In [1]:
import function as f

In [2]:
f.review_cnt_check('수납장')

0번째 item 24240567의 review 수 : 386
1번째 item 14663251의 review 수 : 916
2번째 item 9383538의 review 수 : 170
3번째 item 6782692의 review 수 : 543
4번째 item 14663245의 review 수 : 657
5번째 item 212560의 review 수 : 1064
6번째 item 1509871의 review 수 : 1353
7번째 item 24070993의 review 수 : 354
8번째 item 47423587의 review 수 : 836
9번째 item 858687의 review 수 : 4968
10번째 item 24070996의 review 수 : 540
11번째 item 24070995의 review 수 : 710
12번째 item 6782692의 review 수 : 543
13번째 item 24070993의 review 수 : 354
14번째 item 24240567의 review 수 : 386
15번째 item 122785302의 review 수 : 19
16번째 item 19049940의 review 수 : 91
17번째 item 8789859의 review 수 : 600
18번째 item 63494366의 review 수 : 56
19번째 item 47423587의 review 수 : 836
20번째 item 1509871의 review 수 : 1353
21번째 item 5346978의 review 수 : 221
22번째 item 9932423의 review 수 : 111
23번째 item 63504593의 review 수 : 44
24번째 item 47423587의 review 수 : 836
25번째 item 39128250의 review 수 : 30
No reviews
27번째 item 1013011의 review 수 : 2098
28번째 item 61778368의 review 수 : 35
29번째 item 1013015의 review 수 : 72

In [4]:
%%time
df_train = f.make_df_train('수납장', 0, 10)

총 10개의 상품에서 중복제거 후 9개의 상품 크롤링
10개 상품 중 review가 없는 상품 수 : 0
수납장's Top10 Review Total Count : 9590
수납장's Top10 Filtered Review Count : 5157
Wall time: 6min 40s


In [5]:
display(df_train.tail())

,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
5152,5,1,858687,마켓비 LEITER 600 넓은 4단 선반,레드,2016.03.29,이*선,. 조립두쉽고좋아요,"[(., SF), (조립, NNG), (두, JX), (쉽, VA), (고, ECE...","[조립,쉽,고,좋,]"
5153,5,1,858687,마켓비 LEITER 600 넓은 4단 선반,그레이,2016.03.22,정*경,. 가격이랑 디자인 모두 맘에 듭니다,"[(., SF), (가격, NNG), (이랑, JC), (디자인, NNG), (모두...","[가격,디자인,모두,맘,]"
5154,5,1,858687,마켓비 LEITER 600 넓은 4단 선반,레드,2016.03.19,김*연,. 좋음,"[(., SF), (좋, VA), (음, ETN)]","[좋,]"
5155,5,1,858687,마켓비 LEITER 600 넓은 4단 선반,그레이,2016.03.16,심*수,. 너무좋아요ㅎㅎ 튼튼하구요ㅋ,"[(., SF), (너무, MAG), (좋, VA), (아요, EFN), (ㅎㅎ, ...","[너무,좋,튼튼,하구,ㅋ,]"
5156,4,1,858687,마켓비 LEITER 600 넓은 4단 선반,화이트,2016.01.24,강*훈,. 좋아요 무지,"[(., SF), (좋, VA), (아요, EFN), (무지, NNG)]","[좋,무지,]"


In [84]:
# 크롤링 raw 데이터와 read_csv로 가져온 데이터 concat하지 말 것
# 데이터 타입 다름!!

In [6]:
import csv
df_train.to_csv("storage_0_10.csv", index=False)

In [57]:
df_train_1 = pd.read_csv("keyword1_0_10.csv")
df_train_2 = pd.read_csv("keyword1_10_50.csv")
df_train_3 = pd.read_csv("keyword1_50_100.csv")

In [61]:
df_train_concat = pd.concat([df_train_1, df_train_2, df_train_3]).reset_index()

In [62]:
len(df_train_concat)

13806

In [64]:
# pos_filtered 컬럼에 대해서 아래 작업 필수
df_train_concat['pos_filtered'] = df_train_concat['pos_filtered'].apply(lambda a: a.split("'")[1:-1])
# pos_filtered 컬럼은 하나의 스트링이 한 리스트 안에 들어가 있는 형태이므로
# 스트링의 쿼트를 기준으로 split해서 대괄호와 스트링을 분리한 후 스트링만 리스트에 남긴다

# rating, pos 컬럼도 작업이 필요하지만 pos 컬럼은 완벽하게 작업 현재로서는 불가
df_train_concat['rating'] = df_train_concat['rating'].apply(lambda a: str(a))

In [33]:
# display(df_train_2[i][3742])
# display(df_train_2[i][3742].split("'"))
# display(df_train_2[i][3742].split("'")[1:-1])

"['좋,']"

['[', '좋,', ']']

['좋,']

In [34]:
# col_ls = list(df_train.columns)
# for i in col_ls:
#     display(df_train[i][3742])
#     display(df_train_2[i][3742])
#     display(df_train[i][3742] == df_train_2[i][3742])

In [7]:
X_train = f.Vectorizer_train(df_train, 'Storage', stop_words=['고'])
y_train = df_train['rating_filtered']

In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import pickle

model = MultinomialNB()
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print(classification_report(y_train, y_train_pred))
# pickle.dump(model, open('model_Storage_0_100_5357.pkl', "wb"))

             precision    recall  f1-score   support

          0       0.78      0.68      0.73       533
          1       0.96      0.98      0.97      4624

avg / total       0.94      0.95      0.95      5157



In [9]:
df_test = f.make_df_test("http://www.coupang.com/vp/products/24070993?itemId=93723572&vendorItemId=3165658357&q=%EC%88%98%EB%82%A9%EC%9E%A5&itemsCount=36&searchId=6c94cf7aa1884428bfcc5f39c8e7d63e&rank=7")

제이디퍼니처 로미 3단 서랍장's Review Total Count : 354
제이디퍼니처 로미 3단 서랍장's Filtered Review Count : 177


In [10]:
X_test, voca = f.Vectorizer_test(df_test, 'Storage')
y_test = df_test['rating_filtered']

In [12]:
# model = pickle.load(open("model_BED_0_100_5357.pkl", "rb"))
y_test_pred = model.predict(X_test)
print(classification_report(y_test, y_test_pred))

             precision    recall  f1-score   support

          0       0.83      0.53      0.65        19
          1       0.95      0.99      0.97       158

avg / total       0.93      0.94      0.93       177



In [13]:
f.top_word_negative(X_test, y_test_pred, voca, 15)

,word,count
0,교환,9
1,싸,5
2,물건,5
3,제품,4
4,불량,4
5,너무,4
6,실망,4
7,신청,4
8,가격,3
9,그냥,3


In [14]:
f.top_word_positive(X_test, y_test_pred, voca, 20)

,word,count
0,좋,93
1,서랍,48
2,생각,38
3,작,32
4,딱,30
5,구매,29
6,같,29
7,가격,27
8,잘,23
9,배송,21


In [25]:
f.neg_or_pos(y_test_pred)

(31, 203)

In [ ]:
n-그램 해보기, 위 결과에서 그냥 pos부착해서 명사만 반환하게 해보기